In [3]:
import numpy as np
import pandas as pd
from pandas.io.json import json_normalize
import feather
import json
import pickle

In [28]:
pd.options.display.max_columns = 200

# Reading and Cleaning Data

In [29]:
def convert_json(df):
    json_columns = ['device', 'geoNetwork', 'totals', 'trafficSource']
    for column in json_columns:
        column_as_df = json_normalize(df[column])
        column_as_df.columns = [column+"."+subcolumn for subcolumn in column_as_df.columns]
        df = df.drop(column, axis=1).merge(column_as_df, right_index=True, left_index=True)
    return df

## Date and Time Columns

In [30]:
def date_processing(df):
    df["date"] = pd.to_datetime(df["date"], format = '%Y%m%d')
    df["_weekday"] = df['date'].dt.weekday
    df["_day"] = df['date'].dt.day 
    df["_month"] = df['date'].dt.month
    df["_year"] = df['date'].dt.year
    df["_visitHour"] = pd.to_datetime(df["visitStartTime"], unit = "s").dt.hour
    return df

## Data Type Converions

In [68]:
def df_numeric_bool_fillna(df, is_test_set = False):
    df["totals.visits"] = df["totals.visits"].astype(int)
    df["totals.hits"] = df["totals.hits"].astype(int)
    df["totals.pageviews"].fillna(1, inplace = True)
    df["totals.pageviews"] = df["totals.pageviews"].astype(int)
    df["totals.bounces"].fillna(0, inplace=True)
    df["totals.bounces"] = df["totals.bounces"].astype(int)
    df["totals.newVisits"].fillna(0, inplace=True)
    df["totals.newVisits"] = df["totals.newVisits"].astype(int)
    df["trafficSource.isTrueDirect"].fillna(False, inplace = True)
    df['trafficSource.adwordsClickInfo.isVideoAd'].fillna(True, inplace = True)
    df['totals.timeOnSite'].fillna(df['totals.timeOnSite'].median(skipna = True), inplace = True)
    df['totals.timeOnSite'] = df['totals.timeOnSite'].astype(int)
    df['totals.sessionQualityDim'].fillna(0, inplace=True)
    df['totals.sessionQualityDim'] = df['totals.sessionQualityDim'].astype(int)
    df["totals.transactions"].fillna(0, inplace=True)
    df["totals.transactions"] = df["totals.transactions"].astype(int)
    df["totals.transactionRevenue"].fillna(0.0, inplace=True)
    df["totals.transactionRevenue"] = df["totals.transactionRevenue"].astype(float)
    df["totals.totalTransactionRevenue"].fillna(0.0, inplace=True)
    df["totals.totalTransactionRevenue"] = df["totals.totalTransactionRevenue"].astype(float)
    return df

## Normalisation

Probably only need to log normalise the transaction revenue, but will create a function anyway, just in case

In [69]:
def df_normalise(df, is_test_set):
        df["totals.transactionRevenue"] = df["totals.transactionRevenue"].apply(lambda x: np.log1p(x))
        df["totals.totalTransactionRevenue"] = df["totals.totalTransactionRevenue"].apply(lambda x: np.log1p(x))
        return df

## Constant Columns and hits column?

In [70]:
def df_remove_cols(df, cols_to_drop = []):
    if cols_to_drop == []:
        constant_cols = [col for col in df.columns if df[col].nunique() == 1 and col != "totals.visits"]
        null_cols =  [col for col in df.columns if df[col].isnull().sum()/len(df) > 0.6 and col not in ['totals.sessionQualityDim', 'totals.transactions']] 
        cols_to_drop = constant_cols + null_cols + ['hits','customDimensions']
        df.drop(cols_to_drop, axis = 1, inplace = True)
        return df, cols_to_drop
    else:
        intersection = set(df.columns.tolist()).intersection(cols_to_drop)
        df.drop(intersection, axis = 1, inplace = True)
        return df, cols_to_drop

## Flagging visitor ids as spenders

In [71]:
#def df_flag_spender(df):
#    im_df = pd.DataFrame(df.groupby('fullVisitorId', as_index = False)['totals.transactionRevenue'].sum())
#    im_df.columns = ['fullVisitorId', 'totals.totalTransactionRevenue']
#    im_df['spender'] = np.where(im_df['totals.totalTransactionRevenue']>0.0,True,False)
#    df = df.merge(im_df, on = 'fullVisitorId')
#    return df

In [72]:
def output_df(df, output_file_name):
    feather.write_dataframe(df, output_file_name)
    feather.write_dataframe(df.sample(frac=0.1, random_state = 1), output_file_name.split('.')[0]+"_sample."+output_file_name.split('.')[1])

## Combining PreProcessing Steps

In [73]:
def process(df, is_test_set, cols_to_drop = []):
    
    df = convert_json(df)
    
    df = date_processing(df)
    df = df_numeric_bool_fillna(df, is_test_set)
    
    df = df_normalise(df, is_test_set)
    
    df, cols_to_drop = df_remove_cols(df, cols_to_drop)
    
    return df, cols_to_drop

In [74]:
def df_load(raw_file_name, output_file_name, chunksize, is_test_set = False, cols_to_drop = []):
    
    df_proc = pd.DataFrame()
    json_columns = ['device', 'geoNetwork', 'totals', 'trafficSource']
    df_reader = pd.read_csv(raw_file_name, converters = {column: json.loads for column in json_columns}, dtype = {'fullVisitorId':'str'}, chunksize = chunksize)
    #chunksize works at 100,000. Smaller and not all the columns turn up in the json. 
    #probably a way to fix that later
    
    for chunk_id, df in enumerate(df_reader):
        df.reset_index(drop=True, inplace=True) 
        df, cols_to_drop = process(df, is_test_set, cols_to_drop)
        
        df_proc = pd.concat([df_proc, df], axis = 0, sort=False).reset_index(drop=True)
        
        del df
        
        if chunk_id % 5 == 0:
            print('{}: rows loaded: {}'.format(chunk_id, df_proc.shape[0]))

    if is_test_set:
        output_df(df_proc, output_file_name)
        return df_proc
    else:
        #df_proc = df_flag_spender(df_proc)
        output_df(df_proc, output_file_name)
        pickle.dump(cols_to_drop, open('data/cols_to_drop.pickle', 'wb'))
        return df_proc

In [75]:
%time clean_train_df = df_load('data/train_v2.csv', 'data/clean_train_v2.feather', chunksize = 100000)

0: rows loaded: 100000
5: rows loaded: 600000
10: rows loaded: 1100000
15: rows loaded: 1600000
Wall time: 10min 39s


In [76]:
cols_to_drop = pickle.load(open('data/cols_to_drop_v2.pickle', 'rb'))
clean_test_df = df_load('data/test_v2.csv', 'data/clean_test_v2.feather', chunksize = 100000, is_test_set = True, cols_to_drop = cols_to_drop)

0: rows loaded: 100000


In [4]:
#clean_train_df = feather.read_dataframe('data/clean_train_v2.feather')

Cheating a bit with choosing a large enough chunksize to avoid the issue of the json columns not having every column in each chunk. 
Instead of having null values they just aren't in the json, possibly will need to come back and deal with that. 

## Categorical Encoding

### Investigating

In [78]:
for column in clean_train_df.select_dtypes(include = 'object').columns.tolist():
    print(column + ' ' + str(clean_train_df[column].nunique()))

channelGrouping 8
fullVisitorId 1323730
device.browser 129
device.operatingSystem 24
device.deviceCategory 3
geoNetwork.continent 6
geoNetwork.subContinent 23
geoNetwork.country 228
geoNetwork.region 483
geoNetwork.metro 123
geoNetwork.city 956
geoNetwork.networkDomain 41982
trafficSource.campaign 33
trafficSource.source 345
trafficSource.medium 7
trafficSource.campaignCode 1


In [79]:
(clean_train_df['device.browser'].value_counts()/clean_train_df['device.browser'].value_counts().sum()).head(10)
#top 10

Chrome               0.686665
Safari               0.182730
Firefox              0.037373
Internet Explorer    0.020765
Android Webview      0.020058
Edge                 0.012025
Samsung Internet     0.009244
Opera Mini           0.008791
Safari (in-app)      0.008316
Opera                0.005611
Name: device.browser, dtype: float64

In [80]:
clean_train_df.loc[clean_train_df['totals.transactionRevenue']>0]['device.browser'].value_counts()\
/clean_train_df.loc[clean_train_df['totals.transactionRevenue']>0]['device.browser'].value_counts().sum()

Chrome               0.901858
Safari               0.065356
Firefox              0.018310
Internet Explorer    0.007994
Edge                 0.003997
Opera                0.000756
Safari (in-app)      0.000648
Android Webview      0.000540
Samsung Internet     0.000432
YaBrowser            0.000054
Amazon Silk          0.000054
Name: device.browser, dtype: float64

Chrome accounts for 68% overall, but 95% of spend. Will create group outside top 10 into other and then 1 hot encode

In [81]:
clean_train_df['device.operatingSystem'].value_counts().head(8)
#top 6

Windows          619720
Macintosh        438514
Android          299386
iOS              219334
Linux             63971
Chrome OS         51318
(not set)         11815
Windows Phone      1675
Name: device.operatingSystem, dtype: int64

In [82]:
(100*clean_train_df['geoNetwork.country'].value_counts()/clean_train_df['geoNetwork.country'].value_counts().sum()).head(20)

United States     41.983344
India              6.164884
United Kingdom     4.293122
Canada             2.988696
Germany            2.254590
Japan              2.144600
Brazil             2.074064
Vietnam            2.041108
France             1.890084
Thailand           1.747840
Turkey             1.707274
Taiwan             1.554728
Mexico             1.479216
Australia          1.384973
Spain              1.371568
Netherlands        1.296348
Italy              1.211353
Russia             1.149715
Indonesia          0.976095
Poland             0.929208
Name: geoNetwork.country, dtype: float64

In [83]:
(clean_train_df.loc[clean_train_df['totals.transactionRevenue']>0]['geoNetwork.country'].value_counts()\
/clean_train_df.loc[clean_train_df['totals.transactionRevenue']>0]['geoNetwork.country'].value_counts().sum()).head()
#95% of Spend in the US. Will create a US vs Not US column

United States    0.953765
Canada           0.017014
Venezuela        0.003673
Taiwan           0.001728
Mexico           0.001458
Name: geoNetwork.country, dtype: float64

In [84]:
(clean_train_df.loc[clean_train_df['totals.transactionRevenue']>0]['geoNetwork.region'].value_counts()\
/clean_train_df.loc[clean_train_df['totals.transactionRevenue']>0]['geoNetwork.region'].value_counts().sum()).head()
#will remove

not available in demo dataset    0.384466
California                       0.304904
New York                         0.129794
Illinois                         0.035811
Washington                       0.027655
Name: geoNetwork.region, dtype: float64

In [85]:
(clean_train_df.loc[clean_train_df['totals.transactionRevenue']>0]['geoNetwork.city'].value_counts()\
/clean_train_df.loc[clean_train_df['totals.transactionRevenue']>0]['geoNetwork.city'].value_counts().sum()).head()
#will remove

not available in demo dataset    0.384466
New York                         0.129686
Mountain View                    0.111591
San Francisco                    0.061467
Sunnyvale                        0.045803
Name: geoNetwork.city, dtype: float64

In [86]:
(clean_train_df.loc[clean_train_df['totals.transactionRevenue']>0]['geoNetwork.subContinent'].value_counts()\
/clean_train_df.loc[clean_train_df['totals.transactionRevenue']>0]['geoNetwork.subContinent'].value_counts().sum()).head()
#Will Keep

Northern America    0.970779
South America       0.006590
Eastern Asia        0.004483
Southeast Asia      0.003187
Western Europe      0.002593
Name: geoNetwork.subContinent, dtype: float64

In [87]:
clean_train_df['geoNetwork.networkDomain'].value_counts().head()
#willdrop

(not set)          499049
unknown.unknown    269796
comcast.net         55486
rr.com              28715
verizon.net         26547
Name: geoNetwork.networkDomain, dtype: int64

In [88]:
clean_train_df['trafficSource.campaign'].value_counts().head()
#will drop

(not set)                                                 1604526
Data Share Promo                                            32914
1000557 | GA | US | en | Hybrid | GDN Text+Banner | AS      24410
1000557 | GA | US | en | Hybrid | GDN Remarketing           15149
AW - Dynamic Search Ads Whole Site                          15146
Name: trafficSource.campaign, dtype: int64

In [89]:
clean_train_df['trafficSource.medium'].value_counts()

organic      591783
(none)       565957
referral     432963
cpc           75603
affiliate     32915
cpm            8982
(not set)       134
Name: trafficSource.medium, dtype: int64

### Encoding

#### Train

In [90]:
#clean_train_df['US'] = np.where(clean_train_df['geoNetwork.country'] == 'United States',1, 0)

In [5]:
country_not_top10 = clean_train_df.groupby('geoNetwork.country').count().sort_values('date', ascending = False).index[10:]

In [6]:
device_not_top10 = clean_train_df.groupby('device.browser').count().sort_values('date', ascending = False).index[10:]
clean_train_df['device.browser'].replace(device_not_top10, 'Other', inplace = True)

In [7]:
os_not_top10 = clean_train_df.groupby('device.operatingSystem').count().sort_values('date', ascending = False).index[10:]
clean_train_df['device.operatingSystem'].replace(os_not_top10, 'Other', inplace = True)

In [8]:
clean_prepped_df = pd.concat([clean_train_df, \
                             pd.get_dummies(clean_train_df[['channelGrouping', 'device.browser', 'device.operatingSystem'
                                                            , 'geoNetwork.country','geoNetwork.subContinent', 'trafficSource.medium']])]\
                             , axis = 1)

In [9]:
cat_cols_to_drop = [column for column in clean_train_df.select_dtypes(include = 'object').columns.tolist() if column != 'fullVisitorId']

In [10]:
clean_prepped_df.drop(cat_cols_to_drop, axis = 1, inplace = True)

#### Test

In [11]:
clean_test_df = feather.read_dataframe('data/clean_test_v2.feather')

In [12]:
#clean_test_df['US'] = np.where(clean_test_df['geoNetwork.country'] == 'United States',1, 0)

In [13]:
device_train_values = [value for value in clean_train_df['device.browser'].unique() if value != 'Other']
clean_test_df['device.browser'] = np.where(clean_test_df['device.browser'].isin(device_train_values), clean_test_df['device.browser'], 'Other')

In [14]:
os_train_values = [value for value in clean_train_df['device.operatingSystem'].unique() if value != 'Other']
clean_test_df['device.operatingSystem'] = np.where(clean_test_df['device.operatingSystem'].isin(os_train_values), clean_test_df['device.operatingSystem'], 'Other')

In [15]:
clean_prepped_test_df = pd.concat([clean_test_df, \
                             pd.get_dummies(clean_test_df[['channelGrouping', 'device.browser', 'device.operatingSystem'
                                                            , 'geoNetwork.country','geoNetwork.subContinent', 'trafficSource.medium']])]\
                             , axis = 1)

In [16]:
list(set(clean_prepped_test_df.columns.tolist()).intersection(cat_cols_to_drop))

['geoNetwork.metro',
 'trafficSource.campaign',
 'geoNetwork.continent',
 'geoNetwork.region',
 'trafficSource.source',
 'channelGrouping',
 'device.browser',
 'device.operatingSystem',
 'geoNetwork.networkDomain',
 'geoNetwork.country',
 'device.deviceCategory',
 'geoNetwork.city',
 'geoNetwork.subContinent',
 'trafficSource.medium']

In [17]:
clean_prepped_test_df.drop(list(set(clean_prepped_test_df.columns.tolist()).intersection(cat_cols_to_drop)), axis = 1, inplace = True)

In [18]:
[column for column in clean_prepped_df.columns if column not in clean_prepped_test_df.columns]

['geoNetwork.country_Antigua & Barbuda',
 'geoNetwork.country_Central African Republic',
 'geoNetwork.country_Comoros',
 'geoNetwork.country_Cook Islands',
 'geoNetwork.country_Dominica',
 'geoNetwork.country_Equatorial Guinea',
 'geoNetwork.country_Eritrea',
 'geoNetwork.country_Marshall Islands',
 'geoNetwork.country_Micronesia',
 'geoNetwork.country_Montserrat',
 'geoNetwork.country_Samoa',
 'geoNetwork.country_Sint Maarten',
 'geoNetwork.country_Solomon Islands',
 'geoNetwork.country_St. Helena',
 'geoNetwork.country_St. Martin',
 'geoNetwork.country_St. Pierre & Miquelon',
 'geoNetwork.country_São Tomé & Príncipe',
 'geoNetwork.country_Tonga',
 'geoNetwork.country_Turks & Caicos Islands',
 'geoNetwork.country_Vanuatu',
 'geoNetwork.country_Åland Islands']

In [19]:
[column for column in clean_prepped_test_df.columns if column not in clean_prepped_df.columns]

['geoNetwork.country_Palau']

In [20]:
[column for column in clean_prepped_df.columns if column not in clean_prepped_test_df.columns]

['geoNetwork.country_Antigua & Barbuda',
 'geoNetwork.country_Central African Republic',
 'geoNetwork.country_Comoros',
 'geoNetwork.country_Cook Islands',
 'geoNetwork.country_Dominica',
 'geoNetwork.country_Equatorial Guinea',
 'geoNetwork.country_Eritrea',
 'geoNetwork.country_Marshall Islands',
 'geoNetwork.country_Micronesia',
 'geoNetwork.country_Montserrat',
 'geoNetwork.country_Samoa',
 'geoNetwork.country_Sint Maarten',
 'geoNetwork.country_Solomon Islands',
 'geoNetwork.country_St. Helena',
 'geoNetwork.country_St. Martin',
 'geoNetwork.country_St. Pierre & Miquelon',
 'geoNetwork.country_São Tomé & Príncipe',
 'geoNetwork.country_Tonga',
 'geoNetwork.country_Turks & Caicos Islands',
 'geoNetwork.country_Vanuatu',
 'geoNetwork.country_Åland Islands']

In [21]:
clean_prepped_test_df.drop([column for column in clean_prepped_test_df.columns if column not in clean_prepped_df.columns], axis = 1, inplace = True)

In [22]:
clean_prepped_df.drop([column for column in clean_prepped_df.columns if column not in clean_prepped_test_df.columns], axis = 1, inplace = True)

In [23]:
len(clean_prepped_df.columns), len(clean_prepped_test_df.columns)

(290, 290)

## Revenue and Transactions to Date Fields

In [26]:
clean_prepped_df.columns

Index(['date', 'fullVisitorId', 'visitId', 'visitNumber', 'visitStartTime',
       'device.isMobile', 'totals.visits', 'totals.hits', 'totals.pageviews',
       'totals.bounces',
       ...
       'geoNetwork.subContinent_Western Africa',
       'geoNetwork.subContinent_Western Asia',
       'geoNetwork.subContinent_Western Europe', 'trafficSource.medium_(none)',
       'trafficSource.medium_(not set)', 'trafficSource.medium_affiliate',
       'trafficSource.medium_cpc', 'trafficSource.medium_cpm',
       'trafficSource.medium_organic', 'trafficSource.medium_referral'],
      dtype='object', length=290)

In [49]:
clean_prepped_df.loc[clean_prepped_df['fullVisitorId'] == '3793880875614125844'][['fullVisitorId','date', 'visitStartTime','visitNumber'
                                                                                  , 'totals.transactions', 'totals.transactionRevenue', 'totals.totalTransactionRevenue']]

,fullVisitorId,date,visitStartTime,visitNumber,totals.transactions,totals.transactionRevenue,totals.totalTransactionRevenue
1447782,3793880875614125844,2018-03-13,1520970571,2,0,0.000000,0.000000
1616336,3793880875614125844,2017-09-11,1505164182,1,1,18.599663,18.672217


In [48]:
clean_prepped_test_df.loc[clean_prepped_test_df['fullVisitorId'] == '3793880875614125844'][['fullVisitorId','date', 'visitStartTime','visitNumber'
                                                                                  , 'totals.transactions', 'totals.transactionRevenue', 'totals.totalTransactionRevenue']]

,fullVisitorId,date,visitStartTime,visitNumber,totals.transactions,totals.transactionRevenue,totals.totalTransactionRevenue
379001,3793880875614125844,2018-06-01,1527869629,3,0,0.000000,0.000000
379063,3793880875614125844,2018-06-01,1527881218,4,1,21.678008,21.680699


In [51]:
clean_prepped_df['isTrain'] = 1
clean_prepped_test_df['isTrain'] = 0

In [52]:
full_df = pd.concat([clean_prepped_df, clean_prepped_test_df])

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [79]:
full_df.loc[full_df['fullVisitorId'] == '3793880875614125844'][['fullVisitorId','date', 'visitStartTime','visitNumber'
                                                                                  , 'totals.transactions', 'totals.transactionRevenue', 'totals.totalTransactionRevenue', 'isTrain']].sort_values('visitStartTime')

,fullVisitorId,date,visitStartTime,visitNumber,totals.transactions,totals.transactionRevenue,totals.totalTransactionRevenue,isTrain
1616336,3793880875614125844,2017-09-11,1505164182,1,1,18.599663,18.672217,1
1447782,3793880875614125844,2018-03-13,1520970571,2,0,0.000000,0.000000,1
379001,3793880875614125844,2018-06-01,1527869629,3,0,0.000000,0.000000,0
379063,3793880875614125844,2018-06-01,1527881218,4,1,21.678008,21.680699,0


Want to have the number of previous transations and amount of previous spend detailed

In [56]:
im_df = full_df.loc[full_df['fullVisitorId'] == '3793880875614125844']

In [69]:
max(full_df['visitNumber'])

523

In [84]:
im_df.groupby(['fullVisitorId'], as_index = False)[['totals.transactions', 'totals.transactionRevenue']].rolling(max(full_df['visitNumber']),min_periods = 1).sum()

totals.transactions  totals.transactionRevenue
0 1447782                  0.0                   0.000000
  1616336                  1.0                  18.599663
  379001                   1.0                  18.599663
  379063                   2.0                  40.277671

Still work to do here

In [25]:
feather.write_dataframe(clean_prepped_df, 'data/encode_train_v2.feather')

In [26]:
feather.write_dataframe(clean_prepped_test_df, 'data/encode_test_v2.feather')